<a href="https://colab.research.google.com/github/dionis/sermonLabAI/blob/main/langchain_mvp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ejemplo de RAG utulizando el marco [Langchain](https://python.langchain.com/v0.1/docs/use_cases/question_answering/) para la predición de casas en venta en Cuba**

In [ ]:
import os

if not os.path.exists('data/cuba_house_sale/'):
  !mkdir -p 'data/cuba_house_sale/'

In [ ]:
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

## Descargando y preprocesando la información

In [ ]:
URL = 'https://drive.google.com/file/d/1Nqt7tSiiw6C2KumLoO2QU3qB4vHyKc2E/view?usp=sharing'

file_id = '1Nqt7tSiiw6C2KumLoO2QU3qB4vHyKc2E'
destination = './data/cuba_house_sale/CUBA_HOUSE_SALE.csv'

download_file_from_google_drive(file_id, destination)

In [ ]:
import pandas as pd

dfHouseSale = pd.read_csv(destination)

In [ ]:
dfHouseSale.head(10)

,ID,Title,Content,Excerpt,Date,Post Type,Permalink,Image URL,Image Filename,Image Path,...,Author Last Name,Slug,Format,Template,Parent,Parent Slug,Order,Comment Status,Ping Status,Post Modified Date
0,2925,Calle Hermanos Marín no. 66 e/ Calle 3 y prolo...,NaN,NaN,2024-03-03 21:41:38,properties,http://00.local/property/calle-hermanos-marin-...,NaN,NaN,NaN,...,FFF,calle-hermanos-marin-no-66-e-calle-3-y-prolong...,NaN,NaN,0,0,0,closed,closed,2024-03-03 21:41:38
1,2948,Calle Santa Lucía no. 521. Apto 3 e/ Clarín y ...,DISTRIBUCIÓN: Nivel-1: Garaje Nivel-2: Escaler...,NaN,2024-03-05 16:17:49,properties,http://00.local/property/calle-santa-lucia-no-...,http://00.local/wp-content/uploads/2021/12/01_...,01_resize.png|02_resize.png|03_resize.png|04_r...,C:\Users\alejo\Local Sites\00\app\public/wp-co...,...,FFF,calle-santa-lucia-no-521-apto-3-e-clarin-y-rel...,NaN,NaN,0,0,0,closed,closed,2024-03-05 16:17:49
2,2997,Santa Rita /e San Félix y Carnicería. Centro H...,NaN,NaN,2024-02-09 05:39:18,properties,http://00.local/property/santa-rita-e-san-feli...,http://00.local/wp-content/uploads/2021/12/Pan...,Panorama-1_resize.jpg|a1_resize.jpg|a2_resize....,C:\Users\alejo\Local Sites\00\app\public/wp-co...,...,FFF,santa-rita-e-san-felix-y-carniceria-centro-his...,NaN,NaN,0,0,0,closed,closed,2024-02-09 05:39:18
3,3229,San Agustín no. 418 /e San Francisco y San Ger...,Ideal para vida en familia en el corazón de la...,NaN,2023-02-09 16:04:40,properties,http://00.local/?post_type=properties&p=3229,http://00.local/wp-content/uploads/2021/12/02_...,02_resize.jpg|01_resize.jpg|03_resize.jpg|04_r...,C:\Users\alejo\Local Sites\00\app\public/wp-co...,...,FFF,san-agustin-no-418-e-san-francisco-y-san-geron...,NaN,NaN,0,0,0,closed,closed,2023-12-12 16:30:12
4,3494,San Francisco 570A e/ San Agustín y Barnada. C...,"Ideal para negocio de alquiler, producción de ...",NaN,2024-03-05 15:24:14,properties,http://00.local/property/san-francisco-570a-e-...,http://00.local/wp-content/uploads/2022/01/02....,02.jpg|01.jpg|03.jpg|04.jpg,C:\Users\alejo\Local Sites\00\app\public/wp-co...,...,FFF,san-francisco-570a-e-san-agustin-y-barnada-cen...,NaN,NaN,0,0,0,closed,closed,2024-03-05 15:24:14
5,3507,Santa Lucía /e Reloj y Clarín. Centro Históric...,Ideal para núcleos familiares grandes con gran...,NaN,2024-02-11 17:27:16,properties,http://00.local/property/santa-lucia-e-reloj-y...,http://00.local/wp-content/uploads/2022/01/01-...,01-1.jpg|02-1.jpg|03-1.jpg|04-1.jpg,C:\Users\alejo\Local Sites\00\app\public/wp-co...,...,Chércoles de la Fé,santa-lucia-e-reloj-y-clarin-centro-historico-...,NaN,NaN,0,0,0,closed,closed,2024-02-11 17:27:16
6,4017,Carretera Central Km 4 1/2 Altos de Quintero. ...,"Inmueble Ideal para vivir en familia, con pote...",NaN,2022-03-23 22:31:54,properties,http://00.local/?post_type=properties&p=4017,http://00.local/wp-content/uploads/2022/03/IMG...,IMG_20220322_155950_1.jpg|01-3.jpg|02-2.jpg|03...,C:\Users\alejo\Local Sites\00\app\public/wp-co...,...,FFF,carretera-central-km-4-1-2-altos-de-quintero-s...,NaN,NaN,0,0,0,closed,closed,2023-03-01 20:20:25
7,4063,Calle 13 no. 61A e/ Jardines y Nueva. Veguita ...,Casa ideal para familias medianas o familias q...,NaN,2023-10-24 02:31:52,properties,http://00.local/property/calle-13-no-61a-e-jar...,http://00.local/wp-content/uploads/2022/04/Fac...,Fachada.jpg|01.jpg|02.jpg|IMG_20220402_111609_...,C:\Users\alejo\Local Sites\00\app\public/wp-co...,...,FFF,calle-13-no-61a-e-jardines-y-nueva-veguita-de-...,NaN,NaN,0,0,0,closed,closed,2023-10-24 02:31:52
8,4091,Cristina no. 1273 Altos e/ 11 y A. Mariana de ...,Ideal para familias medianas que desean tener ...,NaN,2023-10-23 20:20:57,properties,http://00.local/?post_type=properties&p=4091,http://00.local/wp-content/uploads/2022/04/IMG...,IMG_20220404_114527_1.jpg|01-1.jpg|02-1.jpg|IM...,C:\Users\alejo\Local Sites\00\app\public/wp-co...,...,FFF,cristina-no-1273-altos-e-11-y-a-mariana-de-la-...,NaN,NaN,0,0,0,closed,closed,2024-02-11 05:03:55
9,4393,Calle 4ta A y B no. 106

Eliminar valores nulos

In [ ]:
dfHouseSale['Content'].fillna("", inplace = True)
dfHouseSale.head(10)

,ID,Title,Content,Excerpt,Date,Post Type,Permalink,Image URL,Image Filename,Image Path,...,Author Last Name,Slug,Format,Template,Parent,Parent Slug,Order,Comment Status,Ping Status,Post Modified Date
0,2925,Calle Hermanos Marín no. 66 e/ Calle 3 y prolo...,,NaN,2024-03-03 21:41:38,properties,http://00.local/property/calle-hermanos-marin-...,NaN,NaN,NaN,...,FFF,calle-hermanos-marin-no-66-e-calle-3-y-prolong...,NaN,NaN,0,0,0,closed,closed,2024-03-03 21:41:38
1,2948,Calle Santa Lucía no. 521. Apto 3 e/ Clarín y ...,DISTRIBUCIÓN: Nivel-1: Garaje Nivel-2: Escaler...,NaN,2024-03-05 16:17:49,properties,http://00.local/property/calle-santa-lucia-no-...,http://00.local/wp-content/uploads/2021/12/01_...,01_resize.png|02_resize.png|03_resize.png|04_r...,C:\Users\alejo\Local Sites\00\app\public/wp-co...,...,FFF,calle-santa-lucia-no-521-apto-3-e-clarin-y-rel...,NaN,NaN,0,0,0,closed,closed,2024-03-05 16:17:49
2,2997,Santa Rita /e San Félix y Carnicería. Centro H...,,NaN,2024-02-09 05:39:18,properties,http://00.local/property/santa-rita-e-san-feli...,http://00.local/wp-content/uploads/2021/12/Pan...,Panorama-1_resize.jpg|a1_resize.jpg|a2_resize....,C:\Users\alejo\Local Sites\00\app\public/wp-co...,...,FFF,santa-rita-e-san-felix-y-carniceria-centro-his...,NaN,NaN,0,0,0,closed,closed,2024-02-09 05:39:18
3,3229,San Agustín no. 418 /e San Francisco y San Ger...,Ideal para vida en familia en el corazón de la...,NaN,2023-02-09 16:04:40,properties,http://00.local/?post_type=properties&p=3229,http://00.local/wp-content/uploads/2021/12/02_...,02_resize.jpg|01_resize.jpg|03_resize.jpg|04_r...,C:\Users\alejo\Local Sites\00\app\public/wp-co...,...,FFF,san-agustin-no-418-e-san-francisco-y-san-geron...,NaN,NaN,0,0,0,closed,closed,2023-12-12 16:30:12
4,3494,San Francisco 570A e/ San Agustín y Barnada. C...,"Ideal para negocio de alquiler, producción de ...",NaN,2024-03-05 15:24:14,properties,http://00.local/property/san-francisco-570a-e-...,http://00.local/wp-content/uploads/2022/01/02....,02.jpg|01.jpg|03.jpg|04.jpg,C:\Users\alejo\Local Sites\00\app\public/wp-co...,...,FFF,san-francisco-570a-e-san-agustin-y-barnada-cen...,NaN,NaN,0,0,0,closed,closed,2024-03-05 15:24:14
5,3507,Santa Lucía /e Reloj y Clarín. Centro Históric...,Ideal para núcleos familiares grandes con gran...,NaN,2024-02-11 17:27:16,properties,http://00.local/property/santa-lucia-e-reloj-y...,http://00.local/wp-content/uploads/2022/01/01-...,01-1.jpg|02-1.jpg|03-1.jpg|04-1.jpg,C:\Users\alejo\Local Sites\00\app\public/wp-co...,...,Chércoles de la Fé,santa-lucia-e-reloj-y-clarin-centro-historico-...,NaN,NaN,0,0,0,closed,closed,2024-02-11 17:27:16
6,4017,Carretera Central Km 4 1/2 Altos de Quintero. ...,"Inmueble Ideal para vivir en familia, con pote...",NaN,2022-03-23 22:31:54,properties,http://00.local/?post_type=properties&p=4017,http://00.local/wp-content/uploads/2022/03/IMG...,IMG_20220322_155950_1.jpg|01-3.jpg|02-2.jpg|03...,C:\Users\alejo\Local Sites\00\app\public/wp-co...,...,FFF,carretera-central-km-4-1-2-altos-de-quintero-s...,NaN,NaN,0,0,0,closed,closed,2023-03-01 20:20:25
7,4063,Calle 13 no. 61A e/ Jardines y Nueva. Veguita ...,Casa ideal para familias medianas o familias q...,NaN,2023-10-24 02:31:52,properties,http://00.local/property/calle-13-no-61a-e-jar...,http://00.local/wp-content/uploads/2022/04/Fac...,Fachada.jpg|01.jpg|02.jpg|IMG_20220402_111609_...,C:\Users\alejo\Local Sites\00\app\public/wp-co...,...,FFF,calle-13-no-61a-e-jardines-y-nueva-veguita-de-...,NaN,NaN,0,0,0,closed,closed,2023-10-24 02:31:52
8,4091,Cristina no. 1273 Altos e/ 11 y A. Mariana de ...,Ideal para familias medianas que desean tener ...,NaN,2023-10-23 20:20:57,properties,http://00.local/?post_type=properties&p=4091,http://00.local/wp-content/uploads/2022/04/IMG...,IMG_20220404_114527_1.jpg|01-1.jpg|02-1.jpg|IM...,C:\Users\alejo\Local Sites\00\app\public/wp-co...,...,FFF,cristina-no-1273-altos-e-11-y-a-mariana-de-la-...,NaN,NaN,0,0,0,closed,closed,2024-02-11 05:03:55
9,4393,Calle 4ta A y B no. 106 e/. S

El texto tiene etiquetas html y debe eliminarse

In [ ]:
dfHouseSale['Content'][15]

'Apartamento ideal para familias grandes o medianas, que buscan un sitio con tranquilidad donde vivir. Tiene hecha la 3ra habitación que comunmente se realiza en estos tipos de apartamentos. Es un apartamento que tiene un notable trabajo de carpintería en puertas y closet lo cual le aporta valor a la decoración del mismo.\r\nNo tiene problemas con el abastecimiento de agua. Se ubica cerca de la parada de el Copa.\r\nEl edificio tiene buen estado de conservación y no presenta filtraciones.\r\n\r\nAdemás se dejan 1 Aire Acondicionado, 5 tanques instalados en el patio y un tanque de fibro ubicado en primer nivel en una zona de concentración de tanques de los vecinos. También el reloj de pared de la sala y la cocina; todas las telas metálicas de las ventanas y el visor con timbre de la puerta principal.\r\n\r\n<strong>NOTA: SÓLO SE ACEPTAN DÓLARES</strong>\r\n\r\n&nbsp;'

In [ ]:
!pip install bs4

In [ ]:
# Import Module
from bs4 import BeautifulSoup

def remove_html_escape(html):
    return BeautifulSoup(str(html), "lxml").text

In [ ]:
dfHouseSale['Content'] = dfHouseSale['Content'].apply(remove_html_escape)

<ipython-input-23-cd15ddcce5b4>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  return BeautifulSoup(str(html), "lxml").text


Transformado y limpio el contenido del texto

In [ ]:
dfHouseSale['Content'][15]

'Apartamento ideal para familias grandes o medianas, que buscan un sitio con tranquilidad donde vivir. Tiene hecha la 3ra habitación que comunmente se realiza en estos tipos de apartamentos. Es un apartamento que tiene un notable trabajo de carpintería en puertas y closet lo cual le aporta valor a la decoración del mismo.\r\nNo tiene problemas con el abastecimiento de agua. Se ubica cerca de la parada de el Copa.\r\nEl edificio tiene buen estado de conservación y no presenta filtraciones.\r\n\r\nAdemás se dejan 1 Aire Acondicionado, 5 tanques instalados en el patio y un tanque de fibro ubicado en primer nivel en una zona de concentración de tanques de los vecinos. También el reloj de pared de la sala y la cocina; todas las telas metálicas de las ventanas y el visor con timbre de la puerta principal.\r\n\r\nNOTA: SÓLO SE ACEPTAN DÓLARES\r\n\r\n\xa0'

Concatenando título y texto para vector de embeddings

In [ ]:
dfHouseSale['Clean_Content'] = dfHouseSale['Title'] + ' _ '  + dfHouseSale['Content']

In [ ]:
dfHouseSale['Clean_Content'][15]

'B 11 Apartamento 5 Antonio Maceo. Santiago de Cuba _ Apartamento ideal para familias grandes o medianas, que buscan un sitio con tranquilidad donde vivir. Tiene hecha la 3ra habitación que comunmente se realiza en estos tipos de apartamentos. Es un apartamento que tiene un notable trabajo de carpintería en puertas y closet lo cual le aporta valor a la decoración del mismo.\r\nNo tiene problemas con el abastecimiento de agua. Se ubica cerca de la parada de el Copa.\r\nEl edificio tiene buen estado de conservación y no presenta filtraciones.\r\n\r\nAdemás se dejan 1 Aire Acondicionado, 5 tanques instalados en el patio y un tanque de fibro ubicado en primer nivel en una zona de concentración de tanques de los vecinos. También el reloj de pared de la sala y la cocina; todas las telas metálicas de las ventanas y el visor con timbre de la puerta principal.\r\n\r\nNOTA: SÓLO SE ACEPTAN DÓLARES\r\n\r\n\xa0'

# Cargar módulos de LangChain para el uso de RAG

In [ ]:
!pip install langchain_community langchain pypdf  tiktoken

# Uso de modelos Open Source de Hugging Face

Los modelos de Hugging Face requieren instalación de einops. Para utilizar `low_cpu_mem_usage = True` o `device_map` es necesario contar con Accelerate instalado: pip install accelerate.

In [ ]:
!pip install -q sentence-transformers transformers einops accelerate

### Dividir documento en Pedazos (Chunk) y Ventana de Contexto

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
    length_function=len
    )



Cargar datos de un Dataframe de Pandas con un **Loader** de LangChain

Nota: Existen ejemplos para PDF, [MongoDB](https://python.langchain.com/v0.1/docs/integrations/document_loaders/mongodb/), [Notion](https://python.langchain.com/v0.1/docs/integrations/document_loaders/notion/), [CSV](https://python.langchain.com/v0.1/docs/integrations/document_loaders/csv/), [GitHub](https://python.langchain.com/v0.1/docs/integrations/document_loaders/github/)

In [ ]:
from langchain_community.document_loaders import DataFrameLoader

loader = DataFrameLoader(dfHouseSale, page_content_column="Clean_Content")

documents = loader.load_and_split(text_splitter)

Instalar los módulos para almacenar los documentos en la Base de Datos Vectorial **ChromaDB**

In [ ]:
!pip install unstructured chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 34.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 6.9 MB/s eta

In [ ]:
len(documents), documents[10].page_content

(126,
 'Calle Heredia no. 72 e/ Corona y Padre Pico. Centro Histórico. Santiago de Cuba _ Vivienda Ideal para familias medianas y grandes; ideal para montar un negocio de renta con vista al mercado internacional; también pudiera ser muy rentable para restaurantes o negocios de elaboración de productos o para el alojamiento de oficinas. Un inmueble con valor arquitectónico, con gran solidez estructural, con privacidad, con amplitud y céntrica posición la convierten sin dudas en una opción excelente para los que quieren triunfar.')

Embeddings e ingesta a base de datos vectorial

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

#Carga de Embedding para obtener vectores

embed_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")



vectorstore = Chroma.from_documents(
    documents = documents,
    embedding = embed_model,
    persist_directory="chroma_db_dir",  # Local mode with in-memory storage only
    collection_name="cuba_house_sale"
)

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3}
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Modelos de chat y cadenas para consulta de información

In [ ]:
from langchain.prompts import PromptTemplate

custom_prompt_template = """Usted es un agente inmobiliario.
Usa la siguiente información para responder a la pregunta del usuario.
Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta.

Contexto: {context}
Pregunta: {question}

Solo devuelve la respuesta útil a continuación y nada más y responde siempre en español
Respuesta útil:
"""
prompt = PromptTemplate(template=custom_prompt_template,
                        input_variables=['context', 'question'])

Plantilla de prompt para [**phi3**](https://www.pragnakalp.com/leverage-phi-3-exploring-rag-based-qna-with-microsofts-phi-3/)

## Prompt templates

Un 'prompt' o 'indicación' es como una receta que le proporcionamos a nuestro modelo de inteligencia artificial (IA). Esta receta contiene los ingredientes y las instrucciones que la IA necesita para cocinar la respuesta que estamos buscando.

1. **Instrucciones:** Esta es la parte donde le decimos a nuestro modelo de IA exactamente qué queremos que haga. Piensa en esto como cuando lees una receta de cocina. Por ejemplo, "corta las verduras", "sofríe los ingredientes", etc.

2. **Información externa o contexto:** Este es el ingrediente que añadimos a nuestra receta. Podría ser información que obtenemos de una base de datos, un cálculo que hemos hecho, etc. Esto le da a nuestro modelo un poco de sabor adicional y contexto sobre lo que estamos buscando.

3. **Entrada del usuario o consulta:** Este es el ingrediente principal de nuestra receta. Es el dato que el usuario introduce y en torno al cual queremos que nuestro modelo cocine la respuesta.

4. **Indicador de salida:** Piensa en esto como el momento en el que sabes que tu receta está lista. Para un modelo que genera código Python, podría ser la palabra 'import', que suele ser el comienzo de muchos scripts de Python. Para un chatbot, podría ser la frase 'Chatbot:', indicando que es hora de que el chatbot hable.

Por lo general, estos componentes se colocan en el orden en que los hemos descrito, igual que seguirías los pasos de una receta de cocina. Empezamos con las instrucciones, añadimos el contexto, luego la entrada del usuario, y finalmente, buscamos nuestro indicador de salida para saber que hemos terminado.

In [ ]:

custom_prompt_template = """<|system|>
Usted es un agente inmobiliario.
Usa la siguiente información para responder a la pregunta del usuario.
Si no sabes la respuesta, simplemente di que no lo sabes, no intentes inventar una respuesta.<|end|>
<|user|>
  Context: {context}
  Question: {question}
<|end|>

  Solo devuelve la respuesta útil a continuación y nada más y responde siempre en español
  Respuesta útil:

<|assistant|>"""
prompt = PromptTemplate(template=custom_prompt_template,
                        input_variables=['context', 'question'])

Cargar el modelo de HugginFace Phi3 o modelo para el Español

In [ ]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

from transformers import AutoTokenizer, pipeline
import torch
model = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model)

llm_pipe = pipeline(
    "text-generation",
    model = model,
    tokenizer = tokenizer,
    torch_dtype = torch.bfloat16,
    trust_remote_code = True,
    device_map = "auto",
    model_kwargs = {
        'temperature': 0.1,
        'max_length': 1420,
        'do_sample': True,
        'top_k': 10,
        'num_return_sequences':1,
        'eos_token_id': tokenizer.eos_token_id
    }
)

llm = HuggingFacePipeline(pipeline = llm_pipe)

tokenizer_config.json:   0%|          | 0.00/3.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

Crear una Cadena de Recuperación para obtener los resultados segun la consulta

Si usamos una `chain_type` *stuff* entonces podemos incluir nuestro propio prompt/plantilla. Sin embargo, le cabe solo el máximo de tokens permitidos por el modelo y no documentos largos.

In [ ]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type = "stuff", # Define la salida
        retriever = retriever,
        return_source_documents = True,
        chain_type_kwargs = {"prompt": prompt}
     )

Otra forma de crear Cadenas (Chain en LangChain)

In [ ]:
#prompt = PromptTemplate.from_template(template)

#chain = prompt | llm

#question = "Cuales opciones de casas tengo de tres cuartos y cerca de un parque?"

#print(chain.invoke({"query": question}))

Consultar al LLM y la Base de Datos Vectorial en modo RAG

In [ ]:
response = qa.invoke({"query": "Cuales opciones de casas tengo de tres cuartos y cerca de un parque?"})
response['result']

### Filtrar la salida o parsearla para que sea mejor la respuesta del LLM

In [ ]:
from langchain.chains.question_answering import load_qa_chain
# Define the QNA chain
chain = load_qa_chain(llm, chain_type="stuff", prompt = prompt)

**Definir una función de ayuda o utilidad para parsear la salida**

Esta función tomará la pregunta del usuario como entrada. Dentro de la función, la pregunta va al recuperador, que compara su incrustación con los documentos almacenados en la base de datos y obtiene el fragmento más relevante. Luego, este fragmento y la pregunta original se pasan a la cadena QnA, que nos da la respuesta.

In [ ]:
# A utility function for answer generation
def ask(question):

   #Obtener los Chunks relevantes a la pregunta en el RAG
   context = retriever.get_relevant_documents(question)
   #print(context)

   answer = (
       chain({
           "input_documents": context,
           "question": question
           },
          return_only_outputs=True)
       )['output_text']

   return answer

Envio de pregunta y obtención de respuestas del RAG y el LLM a partir de la plantilla

In [ ]:
# Take the user input and call the function to generate output
user_question = "¿Cuáles opciones de casas tengo de tres cuartos y cerca de un parque?"

print( f"Question ==> {user_question}")
answer = ask(user_question)

#Paserar la salida en Phi3 siempre el texto que sigue al token "<|assistant|>"
#es la respuesta del LLM asociada al RAG
print("\n\n")

answer = (answer.split("<|assistant|>")[-1]).strip()
print(f"Answer === {answer}")

Question ==> ¿Cuáles opciones de casas tengo de tres cuartos y cerca de un parque?



Answer === La opción de vivienda que cumple con tus criterios de tres cuartos y cerca de un parque es la ubicada en Calle 2 #106, Santiago de Cuba, que es una vivienda urbana ideal para familias medianas, ubicada cerca del parque Carretera del Morro. Sin embargo, esta casa tiene placa ocupada y está interesada en permutarla por una casa similar en los alrededores con placa libre.


Documentos  mas cercanos a la respuesta  en el RAG

In [ ]:
metadata = []
for index, _ in enumerate(response['source_documents']):
    print (f" Document {index}: {_.page_content}")
    #metadata.append(_.metadata)


 Document 0: Calle 2 #106 % 9 y 11. Mariana de la Torre. Santiago de Cuba _ Vivienda urbana ideal para familias medianas, ubicada cerca del parque Carretera del Morro, tiene en su entorno además el consultorio medico y todos los servicios de tienda, gasolinera, etc que se encuentran en esta calle. La casa tiene placa ocupada y desea permutar pars una casa con similares características en los alrededores de la ciudad pero con placa libre.
 Document 1: Calle Indio no. 7 Altos e/ 4 y 7. Mariana de la Torre. Santiago de Cuba _ Inmueble para familias medianas y grandes. Puede ser utilizada como casa de renta para alquiler de habitaciones o de casa completa. Contiene dos niveles totalmente independiente; tiene dos contadores, dos libretas de abastecimiento, dos entradas de agua independiente. Tres tanques de agua elevado y uno bajo. Buena ventilación e iluminación. Tiene un Traspatio en un nivel elevado, el mismo cuenta con árbolles frutales.
 Document 2: Carretera del Morro no.295 e/ 17 y C

## Agentes en LangChain

Los agentes son como "herramientas" para los LLM. Permiten a un LLM acceder a la búsqueda de Google, realizar cálculos complejos con Python e incluso realizar consultas SQL.

In [ ]:
!pip install -qU  sqlalchemy

## ¿Qué es un agente?

**Definición:** La clave detrás de los agentes es brindarles a los LLM la posibilidad de utilizar herramientas en su flujo de trabajo. Esta es una de las principales ventajas de LangChain con respecto a realizar solamente preguntas a un LLM

La definición oficial de agentes es la siguiente:

    Los agentes utilizan un LLM para determinar qué acciones tomar y en qué orden. Una acción puede ser utilizar una herramienta y observar su resultado, o regresar al usuario.

**[Agente para consulta y ejecución de SQL](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/generation/langchain/handbook/06-langchain-agents.ipynb#scrollTo=ny7m_wMCmF4b)**

*Creando una Base de Datos en Memoria*

In [ ]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),
)

In [ ]:
from sqlalchemy import create_engine
from datetime import datetime

engine = create_engine("sqlite:///stocks.db")
metadata_obj.create_all(engine)

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

Función de utilidad para insertar datos en la Base de Datos

In [ ]:
from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0],
    stock_ticker=obs[1],
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

In [ ]:
for obs in observations:
    insert_obs(obs)

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///stocks.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM stocks LIMIT 10;")

sqlite
['stocks']


"[(1, 'ABC', 200.0, '2023-01-01'), (2, 'ABC', 208.0, '2023-01-02'), (3, 'ABC', 232.0, '2023-01-03'), (4, 'ABC', 225.0, '2023-01-04'), (5, 'ABC', 226.0, '2023-01-05'), (6, 'XYZ', 810.0, '2023-01-01'), (7, 'XYZ', 803.0, '2023-01-02'), (8, 'XYZ', 798.0, '2023-01-03'), (9, 'XYZ', 795.0, '2023-01-04'), (10, 'XYZ', 791.0, '2023-01-05')]"

**Tipos de Agentes**

El uso de uno de los agentes prediseñados de langchain implica tres variables:  

*   Definir las herramientas o el conjunto de herramientas
*   Definiendo la llm
*   Definir el tipo de agente

LangChain permite realizar estos requerimientos.

El Agente SQL, del que se puede crear una instancia con su propio método `create_sql_agent`.




Este método utiliza un *toolkit* en lugar de una simple lista de herramientas. Puedes leer más sobre ellos en la [documentación](https://python.langchain.com/docs/modules/agents/toolkits/). Para este caso de uso, usaremos `SQLDatabaseToolkit`.

Como sugiere el nombre, usaremos este agente para realizar tareas de 'zero shot' en la entrada. Eso significa que no tendremos varias interacciones interdependientes sino una sola. Este Agente no tendrá memoria.

¡Ahora estamos listos para inicializar el agente! Usaremos `verbose` en `True` para que podamos ver cuál es el proceso de "pensamiento" ('thinking') de nuestro agente.

**Nota importante:** *Al interactuar con agentes, es muy importante establecer los parámetros `max_iterations` porque los agentes pueden quedarse atrapados en bucles infinitos que consumen muchos tokens. El valor predeterminado es 15 para permitir muchas herramientas y razonamientos complejos, pero para la mayoría de las aplicaciones debes mantenerlo mucho más bajo.*

In [ ]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain


db = SQLDatabase(engine)

agent_executor = create_sql_agent(
    llm = llm,
    toolkit = SQLDatabaseToolkit(db = db, llm = llm),
    verbose = True,
    agent_type = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations = 3
)

chain = create_sql_query_chain( llm, db)

Mensaje de instrucciones (prompts) usado por el Agente SQL:

In [ ]:
print(agent_executor.agent.llm_chain.prompt.template)

AttributeError: 'RunnableAgent' object has no attribute 'llm_chain'

[Para validar los resultados](https://python.langchain.com/v0.1/docs/use_cases/sql/query_checking/). Podemos hacerlo extendiendo la cadena con un segundo mensaje y llamada al modelo:

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

system = """Double check the user's {dialect} query for common mistakes, including:
- Using NOT IN with NULL values
- Using UNION when UNION ALL should have been used
- Using BETWEEN for exclusive ranges
- Data type mismatch in predicates
- Properly quoting identifiers
- Using the correct number of arguments for functions
- Casting to the correct data type
- Using the proper columns for joins

If there are any of the above mistakes, rewrite the query. If there are no mistakes, just reproduce the original query.

Output the final SQL query only."""
prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{query}")]
).partial(dialect=db.dialect)

validation_chain = prompt | llm | StrOutputParser()

full_chain = {"query": chain} | validation_chain

In [ ]:
query = full_chain.invoke(
    {
        "question": "What's the average price from an observations"
    }
)
query

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1256: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'System: Double check the user\'s sqlite query for common mistakes, including:\n- Using NOT IN with NULL values\n- Using UNION when UNION ALL should have been used\n- Using BETWEEN for exclusive ranges\n- Data type mismatch in predicates\n- Properly quoting identifiers\n- Using the correct number of arguments for functions\n- Casting to the correct data type\n- Using the proper columns for joins\n\nIf there are any of the above mistakes, rewrite the query. If there are no mistakes, just reproduce the original query.\n\nOutput the final SQL query only.\nHuman: You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query f

In [ ]:
db.run(query)